In [1]:
import optuna

from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.torch.distributions.distribution_output import NegativeBinomialOutput
from gluonts.dataset.common import MetaData, FileDataset

from model import VQTrEstimator, QuantileLoss, ImplicitQuantileNetworkOutput

In [2]:
 meta = MetaData.parse_file("/home/kashif/.mxnet/gluon-ts/datasets/m5/metadata.json")

In [3]:
train_ds = FileDataset(
        path="/home/kashif/.mxnet/gluon-ts/datasets/m5/train", 
        freq=meta.freq
    )

val_ds = FileDataset(
        path="/home/kashif/.mxnet/gluon-ts/datasets/m5/val", 
        freq=meta.freq,
    )

test_ds = FileDataset(
        path="/home/kashif/.mxnet/gluon-ts/datasets/m5/test", 
        freq=meta.freq,
    )

In [8]:
class VQTrTuningObjective:
    def __init__(
        self, dataset, prediction_length, metric_type="mean_wQuantileLoss"
    ):
        self.dataset = dataset
        self.prediction_length = prediction_length

        self.metric_type = metric_type
        self.tss = None

    def get_params(self, trial) -> dict:
        return {
            #  "context_length":trial.suggest_int("context_length", 240, 336,step = 48),
            #"num_encoder_layers": trial.suggest_int(
            #    "num_encoder_layers", 2, 14, step=4
            #),
            "num_decoder_layers": trial.suggest_int(
                "num_decoder_layers", 2, 14, step=4
            ),
            # "dim_feedforward" :  trial.suggest_int("dim_feedforward", 2, 32,step = 16),
            # "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        }

    def __call__(self, trial):
        params = self.get_params(trial)
        estimator = VQTrEstimator(
            freq=meta.freq,
            prediction_length=meta.prediction_length,
            context_length=meta.prediction_length * 6,
            codebook_size=128,
            dim_head=32,
            nhead=2,
            depth=1,
            num_encoder_layers=2,
            num_decoder_layers=params["num_decoder_layers"],
            dim_feedforward=32,
            activation="gelu",
            
            distr_output=NegativeBinomialOutput(),
            num_feat_dynamic_real=len(meta.feat_dynamic_real),
            num_feat_static_cat=len(meta.feat_static_cat),
            cardinality=[int(cat_feat_info.cardinality) for cat_feat_info in meta.feat_static_cat],
            embedding_dimension = [4, 4, 4, 4, 8],
            
#           num_feat_static_cat=1,
#           cardinality=[int(dataset.metadata.feat_static_cat[0].cardinality)],
#           embedding_dimension=[7],
            
#             distr_output=ImplicitQuantileNetworkOutput("positive", concentration1=0.8, concentration0=0.8),
#             loss=QuantileLoss(),
            
            scaling=True,
            batch_size=256,
            num_batches_per_epoch=200,
            trainer_kwargs=dict(
                max_epochs=60, 
                accelerator="gpu", 
                devices=1,
            ),
        )
        predictor = estimator.train(
            training_data=train_ds,
            validation_data=val_ds,
            num_workers=8,
            shuffle_buffer_length=1024,
            cache_data=True,
        )

        forecast_it, ts_it = make_evaluation_predictions(
            dataset=test_ds, predictor=predictor
        )
        forecasts = list(forecast_it)
        if self.tss is None:
            self.tss = list(ts_it)

        evaluator = Evaluator()
        agg_metrics, _ = evaluator(iter(self.tss), iter(forecasts))
        return agg_metrics[self.metric_type]

In [9]:
import time
start_time = time.time()
study = optuna.create_study(direction="minimize")
study.optimize(
    VQTrTuningObjective(dataset=None, prediction_length=None), 
    n_trials=10
)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
print(time.time() - start_time)

[I 2022-09-11 12:17:22,557] A new study created in memory with name: no-name-12eb4f94-102d-463f-8ee5-28b661db6b56
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | VQTrModel | 107 K 
------------------------------------
107 K     Trainable params
0         Non-trainable params
107 K     Total params
0.432     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Epoch 0, global step 200: 'val_loss' reached 1.50006 (best 1.50006), saving model to '/mnt/scratch/kashif/vq-tr/lightning_logs/version_88/checkpoints/epoch=0-step=200.ckpt' as top 1


...

Validation: 0it [00:00, ?it/s]

Epoch 46, global step 9400: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 47, global step 9600: 'val_loss' was not in top 1
[W 2022-09-11 12:30:41,569] Trial 0 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/home/kashif/.env/pytorch/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_7253/285786193.py", line 72, in __call__
    forecasts = list(forecast_it)
  File "/home/kashif/.env/pytorch/lib/python3.10/site-packages/gluonts-0.9.0.dev0+g643b07b-py3.10.egg/gluonts/torch/model/predictor.py", line 81, in predict
    yield from self.forecast_generator(
  File "/home/kashif/.env/pytorch/lib/python3.10/site-packages/gluonts-0.9.0.dev0+g643b07b-py3.10.egg/gluonts/model/forecast_generator.py", line 173, in __call__
    for batch in inference_data_loader:
  File "/home/kashif/.env/pytorch/lib/python3.10/site-packages/gluonts-0.9.0.dev0+g643b07b-py3.10.egg/gluonts/transform/_base.py", line 103, in __iter__
    yield from 

KeyboardInterrupt: 